🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# Haystack Multi-Agent Yelp Navigator

An educational example of building a multi-agent system using Haystack to create an intelligent Yelp business search assistant.

## What You'll Learn

- How to orchestrate multiple specialized components working together
- How to implement conditional routing with Haystack pipelines
- How to integrate external tools and APIs into Haystack workflows
- How to add quality control through a supervisor pattern

## Architecture

The system has **6 specialized components**:

1. **Clarification Component** - Extracts query, location, and detail level from user input
2. **Search Component** - Finds businesses matching the criteria
3. **Details Component** - Fetches additional info (websites, descriptions) - *optional*
4. **Sentiment Component** - Analyzes customer reviews - *optional*
5. **Summary Component** - Creates a user-friendly response
6. **Supervisor Component** - Reviews quality and can request revisions

And an additional custom component:

7. **StateMultiplexer:** Merges multiple inputs (normal flow + supervisor revisions) into one output, enabling feedback loops.

- [Component implementation](./haystack_helpers/components.py)

The system allows to flexibly choose the best endpoint for the query and routes back if there is incomplete information (determined by the approval node)


## Prerequisites

**Start Hayhooks server:**
```bash
cd yelp-navigator
uv run sh build_all_pipelines.sh && sh start_hayhooks.sh
```

Set `OPENAI_API_KEY` in `.env`

In [ ]:
# =============================================================================
# STEP 1: Imports and Configuration
# =============================================================================

import os
from haystack import Pipeline
from haystack.components.agents.state import State  # Official Haystack State
from haystack.dataclasses import ChatMessage
from haystack_helpers.components import (ClarificationComponent, StateMultiplexer,
                                        SearchComponent, DetailsComponent, SentimentComponent, SummaryComponent,
                                        SupervisorComponent)
from haystack_helpers.state import YELP_AGENT_STATE_SCHEMA, create_yelp_state

from dotenv import load_dotenv


# Load environment
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY is not set. The LLM components will fail.")
else:
    print("✅ OpenAI API key configured")

# Hayhooks Configuration
BASE_URL = "http://localhost:1416"
print(f"✅ Hayhooks server: {BASE_URL}")

## Multi-Agent Supervisor Pattern

Using Haystack to build stateful, cyclic workflows with components.

### Key Concepts

- **State**: Automatic message history handling with schema validation
- **Conditional Routing**: Dynamic workflows based on component outputs
- **Supervisor Pattern**: Quality control that reviews outputs and requests revisions

See [State](./haystack_helpers/state.py) For implementation details.

## State Management

`State` provides automatic message handling with schema validation:
- Appends new messages to history
- Validates all field types
- Supports custom merge handlers

**Key Methods:** `state.get("key", default)`, `state.set("key", value)`, `state.has("key")`

**Custom Fields:** `user_query`, `clarified_query`, `clarified_location`, `detail_level`, `search_results`, `details_data`, `sentiment_data`, `final_summary`

## Step 4: Building the Pipeline Graph

Creating a **cyclic pipeline** with conditional routing and feedback loops.

### Key Components

**Multiplexers:** Handle multiple inputs (normal flow + supervisor revisions)  
**Workers:** Perform actual work with multiple outputs for conditional routing  
**Feedback Loops:** Supervisor sends state back to any previous component

### Connection Pattern

```python
# Forward flow
pipe.connect("clarifier.state", "search_mux.source_1")
pipe.connect("search_mux.state", "searcher.state")

# Conditional routing via component outputs
pipe.connect("searcher.to_details", "details_mux.source_1")
pipe.connect("searcher.to_summary", "summary_mux.source_1")

# Feedback loop
pipe.connect("supervisor.revise_search", "search_mux.source_2")
```

In [ ]:
# =============================================================================
# STEP 4: Building the Pipeline Graph
# =============================================================================

def build_pipeline():
    pipe = Pipeline()

    # 1. Add Components
    pipe.add_component("clarifier", ClarificationComponent())
    
    # Multiplexers (The traffic cops for many-to-one connections)
    pipe.add_component("search_mux", StateMultiplexer("Search Mux"))
    pipe.add_component("details_mux", StateMultiplexer("Details Mux"))
    pipe.add_component("sentiment_mux", StateMultiplexer("Sentiment Mux"))
    pipe.add_component("summary_mux", StateMultiplexer("Summary Mux"))
    
    # Workers
    pipe.add_component("searcher", SearchComponent())
    pipe.add_component("detailer", DetailsComponent())
    pipe.add_component("sentiment", SentimentComponent())
    pipe.add_component("summarizer", SummaryComponent())
    pipe.add_component("supervisor", SupervisorComponent())

    # 2. Connect the "Happy Path" (Left to Right)
    
    # Start -> Clarifier -> Search Mux -> Searcher
    pipe.connect("clarifier.state", "search_mux.source_1")
    pipe.connect("search_mux.state", "searcher.state")
    
    # Searcher -> (Details Mux OR Summary Mux)
    pipe.connect("searcher.to_details", "details_mux.source_1")
    pipe.connect("searcher.to_summary", "summary_mux.source_1")
    
    # Details Mux -> Detailer -> (Sentiment Mux OR Summary Mux)
    pipe.connect("details_mux.state", "detailer.state")
    pipe.connect("detailer.to_sentiment", "sentiment_mux.source_1")
    pipe.connect("detailer.to_summary", "summary_mux.source_2")
    
    # Sentiment Mux -> Sentiment -> Summary Mux
    pipe.connect("sentiment_mux.state", "sentiment.state")
    pipe.connect("sentiment.to_summary", "summary_mux.source_3")
    
    # Summary Mux -> Summarizer -> Supervisor
    pipe.connect("summary_mux.state", "summarizer.state")
    pipe.connect("summarizer.state", "supervisor.state")

    # 3. Connect the "Feedback Loops" (Right to Left / Cycles)
    # Supervisor outputs connect back to the Multiplexers of previous nodes
    
    pipe.connect("supervisor.revise_search", "search_mux.source_2")
    pipe.connect("supervisor.revise_details", "details_mux.source_2")
    pipe.connect("supervisor.revise_sentiment", "sentiment_mux.source_2")
    pipe.connect("supervisor.revise_summary", "summary_mux.source_4")

    return pipe

## Pipeline Visualization

Shows components, connections, conditional routing points, and feedback loops (clarification loop, supervisor revisions).

![](./images/haystack_graph.png)

In [ ]:
# =============================================================================
# STEP 5: Execution
# =============================================================================

if __name__ == "__main__":
    pipeline = build_pipeline()
    
    # Visualize the pipeline structure (optional, requires graphviz)
    pipeline.draw(path="haystack_graph.png")
    
    # Initialize query
    query = "Find me pizza places in Chicago"
    print(f"🚀 Starting Pipeline with query: {query}\n")
    
    # Run pipeline
    # The clarifier component will create the initial State object internally
    results = pipeline.run(
        {"clarifier": {"query": query}}
    )
    
    # Extract final state from supervisor's approved output
    final_state = results["supervisor"]["approved"]
    
    # Display results using Haystack State methods
    print(f"\n{'='*60}")
    print(f"FINAL RESULT: {final_state.get('final_summary')}")
    print(f"{'='*60}")
    
    # Show state statistics
    print(f"\n📊 State Statistics:")
    print(f"  • User Query: {final_state.get('user_query')}")
    print(f"  • Clarified Query: {final_state.get('clarified_query')}")
    print(f"  • Location: {final_state.get('clarified_location')}")
    print(f"  • Detail Level: {final_state.get('detail_level')}")
    print(f"  • Messages: {len(final_state.get('messages', []))}")
    print(f"  • Approval Attempts: {final_state.get('approval_attempts', 0)}")
    
    # Access message history (automatically tracked by State)
    messages = final_state.get('messages', [])
    if messages:
        print(f"\n💬 Message History ({len(messages)} messages):")
        for i, msg in enumerate(messages[-3:], 1):  # Show last 3
            print(f"  {i}. [{msg.role}]: {msg.text[:80]}...")

🚀 Starting Pipeline with query: Find me pizza places in Chicago


🧠 [Clarification] Processing: Find me pizza places in Chicago
🔀 [Search Mux] Forwarding state...

🔍 [Search] Looking for: pizza places in Chicago
🔀 [Summary Mux] Forwarding state...

📝 [Summary] Generating summary...
👮 [Supervisor] Reviewing (Attempt 1)...
✅ [Supervisor] Approved!

FINAL RESULT: If you're in Chicago and on the hunt for some delicious pizza, you're in for a treat! The city is home to some incredible pizza joints that cater to all tastes. For a slice of heaven, head over to Pequod's Pizza, which boasts a solid 3.9-star rating from nearly 9,000 reviews. It's a fan favorite thanks to its cozy atmosphere and tasty offerings. 

Another popular spot is Giordano's, clocking in with a 3.8-star rating. Known for its famous stuffed deep-dish pies, it's a must-visit for anyone eager to try authentic Chicago-style pizza. If you're looking for a pizzeria with rave reviews, Lou Malnati's Pizzeria can't be beaten, boast

## Key Features

- Extracts query, location, detail level
- Adaptive workflow based on user needs
- Calls Haystack pipelines via Hayhooks
- Supervisor reviews with max 2 attempts
- Supports `general`, `detailed`, or `reviews` detail levels

### How It Works

1. Query enters through `clarifier`
2. State propagates through components
3. Components choose outputs based on state (`detail_level`)
4. Supervisor approves or sends back via multiplexer
5. Components track attempts to prevent infinite loops

`pipeline.draw()` visualizes all components, connections, and feedback loops.

## Design Patterns

**Component Architecture:** Custom components with multiple outputs for conditional routing  
**StateMultiplexer:** Enables feedback loops by merging multiple inputs  
**Schema Validation:** Haystack State ensures type safety  
**Supervisor Approval:** Quality control with feedback loops
